### קוד מבוא

#### ספריות

In [4]:
import os
import pandas as pd
import sys

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

#### העלאת משתנים להרצת הקוד

In [6]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

future_folder = os.path.dirname(software_data_folder_location)

sys.path.append(future_folder)

### פונקציות

### פונקציות גלובליות

In [7]:
from global_functions import up_load_shp, up_load_df, split_index_by_taz, change_cols_names

### העלאת קבצים

#### קיבולת ברמת יישוב (חלקי) ביו"ש

##### העלאת טבלה

In [8]:
kibolt_yosh_some_towns=up_load_df(r'{}\background_files'.format(software_data_folder_location),'230628_Jewish_kibolt_some_towns_in_yosh_till_2050')

In [9]:
col=[ 'Taz_num', 'add_aprt_2020__2025',
 'add_aprt_2025__2030',
 'add_aprt_2030__2035',
 'add_aprt_2035__2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns.pivot_table(index='Taz_num',aggfunc=sum).reset_index()[col]

kibolt_yosh_some_towns_sum_by_taz['kibolet_type']='Shrinkable'

C:\Users\dpere\AppData\Local\Temp\ipykernel_26700\702667703.py:8: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns.pivot_table(index='Taz_num',aggfunc=sum).reset_index()[col]


In [10]:
old_col=[ 'Taz_num','add_aprt_2020__2025',
 'add_aprt_2025__2030',
 'add_aprt_2030__2035',
 'add_aprt_2035__2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050','kibolet_type']

new_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

kibolt_yosh_some_towns_sum_by_taz=change_cols_names(kibolt_yosh_some_towns_sum_by_taz, old_col, new_col)

kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns_sum_by_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

kibolt_yosh_some_towns_sum_by_taz['source']='by_town'

#### קיבולת ברמת אזור תנועה (חלקי)

##### העלאת שכבה

In [11]:
Jewish_kibolt_some_by_taz=up_load_shp(r'{}\background_files\kibolt_jew_by_some_taz_230628.shp'.format(software_data_folder_location))

In [12]:
old_col=[ 'Taz_num','add_aprt_2',
 'add_aprt_3',
 'add_aprt_4',
 'add_aprt_5',
 'add_aprt_6',
 'add_aprt_7','type_kibol']

new_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

Jewish_kibolt_some_by_taz=change_cols_names(Jewish_kibolt_some_by_taz, old_col, new_col)

Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==1,'kibolet_type']='Shrinkable'

Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==2,'kibolet_type']='left_the_station'

Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==3,'kibolet_type']='Depends_on_policy'

doubling_aprt=[7310,7711]

col=['2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050']

Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['Taz_num'].isin(doubling_aprt),col]=Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['Taz_num'].isin(doubling_aprt)][col]*2

Jewish_kibolt_some_by_taz=Jewish_kibolt_some_by_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

Jewish_kibolt_some_by_taz['source']='by_taz'

C:\Users\dpere\AppData\Local\Temp\ipykernel_26700\3573415710.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Shrinkable' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==1,'kibolet_type']='Shrinkable'


#### קיבולת ברמת תכנית עם אזורי תנועה בלבד (חלקי)

##### העלאת שכבה

In [13]:
Jewish_kibolt_some_by_plan_only_with_taz=up_load_shp(r'{}\background_files\Jewish_kibolt_some_by_plan_with_taz_num.shp'.format(software_data_folder_location))

In [14]:
Jewish_kibolt_some_by_plan_only_with_taz=Jewish_kibolt_some_by_plan_only_with_taz[['Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_ty']].rename(columns={'kibolet_ty':'kibolet_type'})

Maale_Adumim_extension=[7025, 7028, 7027, 7026]

Jewish_kibolt_some_by_plan_only_with_taz.loc[Jewish_kibolt_some_by_plan_only_with_taz['Taz_num'].isin(Maale_Adumim_extension),col]=Jewish_kibolt_some_by_plan_only_with_taz.loc[Jewish_kibolt_some_by_plan_only_with_taz['Taz_num'].isin(Maale_Adumim_extension)][col]*1.25

Jewish_kibolt_some_by_plan_only_with_taz=Jewish_kibolt_some_by_plan_only_with_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

Jewish_kibolt_some_by_plan_only_with_taz['source']='plan_only_with_taz'

#### קיבולת ברמת תכנית  (חלקי)

##### העלאת טבלאות ושכבות

In [15]:
df=up_load_df(r'{}\background_files'.format(software_data_folder_location),'bau_‏‏index_for_id_polygons_230712')

shp=up_load_shp(r'{}\background_files\230628_id_polygons_for_index.shp'.format(software_data_folder_location))

taz=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))

c:\Users\dpere\miniconda3\envs\creat_forecast_ad_hoc_env\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [16]:
index=shp.merge(df,on='id',how='outer')

#לחלק את התכניות לפי אזורי תנועה
index=split_index_by_taz(index,taz,0.25,['add_aprt'])

#לייצר קיבולת כל חומש
years=[ '2025', '2030', '2035', '2040', '2045', '2050']

index[years]=index[years].apply(lambda x: x * index['risk_factor']*index['add_aprt'])

col=['Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

Jewish_kibolt_some_by_plan=index[col].fillna(0).melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

Jewish_kibolt_some_by_plan['source']='by_plan'

c:\Users\dpere\miniconda3\envs\creat_forecast_ad_hoc_env\Lib\site-packages\geopandas\geodataframe.py:2469: UserWarning: `keep_geom_type=True` in overlay resulted in 13 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


### מכאן  צריך לחבר ביחד את כל הטבלאות

In [17]:
kibolt_jew=pd.concat([Jewish_kibolt_some_by_plan,Jewish_kibolt_some_by_plan_only_with_taz,kibolt_yosh_some_towns_sum_by_taz,Jewish_kibolt_some_by_taz],axis=0)

kibolt_jew['year']=kibolt_jew['year'].astype(str)

In [18]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

### לשנות קיבולת

כדי לשנות קיבולת צריך לעשות את זה לפי מגזר

##### העלאת טבלה

In [19]:
folder_path=r'{}\background_files'.format(software_data_folder_location)

taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')

In [20]:
lst_change_sector=list(taz_sector_change.Taz_num)

taz.loc[(taz['Taz_num'].isin(lst_change_sector)),'main_secto']='U_Orthodox'

In [21]:
kibolt_jew=kibolt_jew.merge(taz[['Taz_num','main_secto','zonetype']],on='Taz_num',how='left')

שינוי בקיבולת לפי מגזר

In [22]:
kibolt_jew=kibolt_jew.loc[~((kibolt_jew['main_secto']=="Jewish")&
                            (kibolt_jew['zonetype']=="Jerusalem")&
                            (kibolt_jew['kibolet_type']=="Depends_on_policy"))]

In [23]:
kibolt_jew.loc[(kibolt_jew['main_secto']=="Jewish")&
               (kibolt_jew['zonetype']=="Jerusalem")&
               (kibolt_jew['kibolet_type']=="Shrinkable"),'add_aprt']=kibolt_jew['add_aprt']*0.27

### ייצוא לקלט של מכונת התפלגות גילים תרחיש דיור

In [24]:
kibolt_jew=kibolt_jew.pivot_table(index='Taz_num',columns='year',values='add_aprt',aggfunc=sum).fillna(0).reset_index()

C:\Users\dpere\AppData\Local\Temp\ipykernel_26700\3151052792.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  kibolt_jew=kibolt_jew.pivot_table(index='Taz_num',columns='year',values='add_aprt',aggfunc=sum).fillna(0).reset_index()


In [25]:
old_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050']

new_col=[ 'Taz_num','add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

kibolt_jew=kibolt_jew[old_col]

kibolt_jew.columns=new_col

save_file=r'{}\Intermediates'.format(software_data_folder_location)

kibolt_jew.to_excel('{}\{}_kibolt_jew_till_2050.xlsx'.format(save_file,file_date),index=False)